----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content - Pricing - Streaming Events

This notebook demonstrates using a Pricing object to receive streaming real-time data via the event driven interface.  

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fdd732222b0 {name='workspace'}>

### Define a callback to receive streaming data

In [4]:
# Function to handle the intial Refresh for each item
def handle_refresh(fields, instrument_name, streaming_prices):
    # One way to access data - get dataframe
    print(f"Refresh : {streaming_prices.get_snapshot()}")
    return

# Function to update dataframe, when we receive updates for individual items
def handle_update(fields, instrument_name, streaming_prices):
    # Alternative way of accesing data - access the updated fields
    print(f"Update : {instrument_name}:{fields}")
    
# Function to extract status code for an item as & when received from server
# Status contains a 'code' and a more detailed 'message'
def handle_status(status, instrument_name, streaming_prices):
    state=status['State']
    print(f"Status : {instrument_name}:{state['Code']}:{state['Text']}")

### Create streams for our RICs and open them

In [5]:
# Define our Streaming Price object - including a bad ric for illustrative purposes
streams = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()

# Callback for if we wanted to handle invidiual Refresh for each item
streams.on_refresh(handle_refresh)
# Specify callback handler for any updates
streams.on_update(handle_update)
# Specify callback handler for any updates
streams.on_status(handle_status)

# Send the requests to the server and open the streams for all items
streams.open()

Refresh :   Instrument     ASK     BID
0       EUR=  1.0596  1.0592
1       GBP=    <NA>    <NA>
2       JPY=    <NA>    <NA>
3     BADRIC    <NA>    <NA>
Refresh :   Instrument     ASK     BID
0       EUR=  1.0596  1.0592
1       GBP=  1.2306  1.2302
2       JPY=    <NA>    <NA>
3     BADRIC    <NA>    <NA>
Refresh :   Instrument     ASK     BID
0       EUR=  1.0596  1.0592
1       GBP=  1.2306  1.2302
2       JPY=  135.87  135.84
3     BADRIC    <NA>    <NA>
Status : BADRIC:NotFound:The record could not be found


<OpenState.Opened: 'Opened'>

Update : GBP=:{'ASK': 1.2305, 'BID': 1.2304}
Update : JPY=:{'ASK': 135.86, 'BID': 135.85}
Update : EUR=:{'ASK': 1.0595, 'BID': 1.0594}
Update : JPY=:{'ASK': 135.86, 'BID': 135.85}
Update : EUR=:{'ASK': 1.0596, 'BID': 1.0592}
Update : JPY=:{'ASK': 135.89, 'BID': 135.84}
Update : EUR=:{'ASK': 1.0595, 'BID': 1.0593}
Update : JPY=:{'ASK': 135.86, 'BID': 135.85}
Update : GBP=:{'ASK': 1.2305, 'BID': 1.2304}
Update : JPY=:{'ASK': 135.86, 'BID': 135.85}
Update : JPY=:{'ASK': 135.86, 'BID': 135.85}
Update : EUR=:{'ASK': 1.0596, 'BID': 1.0592}
Update : GBP=:{'ASK': 1.2306, 'BID': 1.2302}
Update : JPY=:{'ASK': 135.86, 'BID': 135.83}
Update : EUR=:{'ASK': 1.0594, 'BID': 1.0593}
Update : GBP=:{'ASK': 1.2304, 'BID': 1.2303}


## Close the stream

In [6]:
streams.close()

Update : JPY=:{'ASK': 135.86, 'BID': 135.85}
Update : EUR=:{'ASK': 1.0595, 'BID': 1.0593}


<OpenState.Closed: 'Closed'>

## Close the session

In [7]:
rd.close_session()